### 1. 连接百度

In [4]:
'''
    qazyhn	 Qaz123
'''

from aip import AipNlp

APP_ID = '********'
API_KEY = '*************'
SECRET_KEY = '*************'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

### 2. 加载数据

In [1]:
count = 0
alldata = []
with open("./all_docs.txt", "r", encoding="utf-8") as data:
    for line in data:
        line = line.strip()
        alldata.append(line)
        count += 1 
print(count)
print(len(alldata))

108295
108295


In [2]:
'''
    只读取id
    只读取title + content
    组成一个id：title+"。"+content的词典
'''
train_id_list = []
train_title_list = []
train_title_content_list = []
train_dict = {}  # key = id, value = title+"。"+content
for line in alldata:
    line = line.strip()
    if len(line.split("\001")) == 3:
        ids = line.split("\001")[0]
        title = line.split("\001")[1]
        title_content = line.split("\001")[1] + "。" + line.split("\001")[2]
        
        train_id_list.append(ids)
        train_title_list.append(title)
        train_title_content_list.append(title_content)
        train_dict[ids] = title_content

In [3]:
print(len(train_id_list))
print(len(train_title_list))
print(len(train_title_content_list))
print(len(train_dict))

108295
108295
108295
108295


### 3. 将所有的titles分词

In [5]:
def cutSentences(text):
    
    import time
    
    cutwords = []

    if text is None:
        return None

    result = client.lexerCustom(text)

    items = result.get('items', [])
    items_size = len(items)

    tries_limit = 3
    tries_counter = 0
    
    while items_size == 0: # 分词结果为空
        if len(text) != 0: # 但句子长度不为空
            time.sleep(1)
            result = client.lexerCustom(text)

            items = result.get('items', [])
            items_size = len(items)

        tries_counter += 1

        if tries_counter >= tries_limit: # 分词尝试大于等于4次之后仍失败
            return None
        
    restore_idx = 0
    
#     while restore_idx < items_size: # 对每个分词结果进行整理 
    
    for dic in items:
        if dic['pos'] not in ["d", "m", "q", "r", "p", "c", "u", "xc", "w"]:
            cutwords.append(dic['item'])
    
    return cutwords

# d副词
# m数量词	q量词	r代词	p介词
# c连词	u助词	xc其他虚词	w标点符号

In [6]:
import re
with open("./getAllTitlesCutData.txt", "w", encoding="utf-8") as result:
    for num, sent in enumerate(train_title_list): # 每篇文档的title      
        sent = sent.encode('gb2312', 'ignore').decode('gb2312').encode('gbk', 'ignore').decode('gbk').strip()
        try:
            result.write(" ".join(cutSentences(sent))+"\n")
        except Exception as e:
            print(num)
            print(e)

25489
can only join an iterable
